In [1]:
from keras import layers
from keras import models

Using TensorFlow backend.


In [2]:
def build_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPool2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    return model 

In [3]:
model = build_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [4]:
from keras.datasets import mnist
from keras.utils import to_categorical
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

import numpy as np
def add_dim_1 (shape):
    return np.concatenate( [np.array(shape), np.array((1,))] )
tis = add_dim_1(train_images.shape)
print(tis)
train_images = train_images.reshape(add_dim_1(train_images.shape))
test_images = test_images.reshape(add_dim_1(test_images.shape))
print(train_images.shape)
print(test_images.shape)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
print(train_labels.shape, test_labels.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)
[60000    28    28     1]
(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000, 10) (10000, 10)


In [5]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 6s 108us/step - loss: 14.4314 - acc: 0.1045
Epoch 2/5
60000/60000 [==============================] - 3s 48us/step - loss: 14.4351 - acc: 0.1044
Epoch 3/5
60000/60000 [==============================] - 3s 48us/step - loss: 14.4351 - acc: 0.1044
Epoch 4/5
60000/60000 [==============================] - 3s 48us/step - loss: 14.4351 - acc: 0.1044
Epoch 5/5
60000/60000 [==============================] - 3s 48us/step - loss: 14.4351 - acc: 0.1044


In [6]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_loss, test_acc)

10000/10000 [==============================] - 0s 35us/step
14.461155032348632 0.1028


In [7]:
import matplotlib.pyplot as plt
idx = 12
print(test_labels[idx])
print(test_images[idx].shape)
print(add_dim_1(test_images[idx].shape))
plt.imshow(test_images[idx].reshape(test_images[idx].shape[0:2]), cmap='gray')
pred_ar = test_images[idx:idx+1]
print(pred_ar.shape)
print(pred_ar[0].shape)
plt.figure()
plt.imshow(pred_ar[0].reshape(pred_ar[0].shape[0:2]), cmap='gray')
pred_probs = model.predict(pred_ar)
print(pred_probs)
print(np.argmax(pred_probs))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
(28, 28, 1)
[28 28  1  1]
(1, 28, 28, 1)
(28, 28, 1)
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
7


In [22]:
# kaggle competitions download -c dogs-vs-cats
import os, shutil
original_dataset_dir = '/home/ktdiedrich/.kaggle/competitions/dogs-vs-cats'
base_dir = '/home/ktdiedrich/Documents/localData/machineLearning/dogs-vs-cats_small'
if not os.path.exists(base_dir):
    os.makedirs(base_dir, exist_ok=True)
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
train_cats_dir = os.path.join
validation_dir = os.path.join(base_dir, 'validation')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

dir_paths = [train_cats_dir, train_dogs_dir, 
        validation_cats_dir, validation_dogs_dir, test_cats_dir, test_dogs_dir]

for dir_path in dir_paths: 
    if not os.path.exists(dir_path):
        os.makedirs(dir_path, exist_ok=True)
import cv2
def copy_files(base_name, destination, begin, end, source=original_dataset_dir):
    fnames = ['{}.{}.jpg'.format(base_name, i) for i in range(begin, end)]
    for fname in fnames:
        src = os.path.join(source, fname)
        img = cv2.imread(src)
        # print(img.shape)
        img = cv2.resize(img, (150, 150))
        dst = os.path.join(destination, fname)
        cv2.imwrite(dst, img)
        #shutil.copyfile(src, dst)
        
data_set_dir = os.path.join(original_dataset_dir, 'train')
copy_files(base_name='cat', destination=train_cats_dir, begin=0, end=1000, source=data_set_dir)
copy_files(base_name='dog', destination=train_dogs_dir, begin=0, end=1000, source=data_set_dir)

copy_files(base_name='cat', destination=validation_cats_dir, begin=1000, end=1500, source=data_set_dir)
copy_files(base_name='dog', destination=validation_dogs_dir, begin=1000, end=1500, source=data_set_dir)

copy_files(base_name='cat', destination=test_cats_dir, begin=1500, end=2000, source=data_set_dir)
copy_files(base_name='dog', destination=test_dogs_dir, begin=1500, end=2000, source=data_set_dir)

for dir_path in dir_paths:
    print("{}: {}".format(dir_path, len(os.listdir(dir_path))))

/home/ktdiedrich/Documents/localData/machineLearning/dogs-vs-cats_small/train/cats: 1000
/home/ktdiedrich/Documents/localData/machineLearning/dogs-vs-cats_small/train/dogs: 1000
/home/ktdiedrich/Documents/localData/machineLearning/dogs-vs-cats_small/validation/cats: 500
/home/ktdiedrich/Documents/localData/machineLearning/dogs-vs-cats_small/validation/dogs: 500
/home/ktdiedrich/Documents/localData/machineLearning/dogs-vs-cats_small/test/cats: 500
/home/ktdiedrich/Documents/localData/machineLearning/dogs-vs-cats_small/test/dogs: 500
